In [1]:
import os
os.chdir("../../")

In [2]:
import sqlite3
import pandas as pd
from api.utils.database import rows_to_dicts

In [3]:
con = sqlite3.connect("./pipeline/database.db")

In [4]:
cur = con.cursor()
rows = cur.execute("""
SELECT *
FROM rideshare
LIMIT 5
""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,week,pickup_community_area,dropoff_community_area,n_trips,n_trips_pooled_authorized,n_trips_pooled,avg_cost_no_tip_cents
0,2018-11-04,1,1,2223,701,260,627
1,2018-11-04,52,28,20,14,8,2567
2,2018-11-04,52,24,3,2,1,3228
3,2018-11-04,52,22,1,0,0,6800
4,2018-11-04,52,20,1,1,1,2260


# Notes
* One row per community area
* Each row has area info, 2019 disability rate, number of trips per day before covid, num trips per day after covid
* 3 Tables to Join: community areas, disability rates, rideshare per community area
* Before = March 2019 - Feb 2020
* After =  March 2020 - Feb 2021



In [9]:
cur = con.cursor()
rows = cur.execute("""
SELECT 
    c.name,
    c.part,
    c.population,
    c.area_number,
    r.*,
    d.*
FROM community_area c
    LEFT JOIN (
        SELECT 
            pickup_community_area,
            SUM(CASE WHEN week < '2021-03-01' THEN n_trips ELSE 0 END) / 365 AS avg_trips_before,
            SUM(CASE WHEN week >= '2020-03-01' THEN n_trips ELSE 0 END) / 365 AS avg_trips_since
        FROM rideshare
        WHERE 
            week >= '2019-03-01'
            AND week < '2021-03-01'
        GROUP BY 
            pickup_community_area 
    ) r
        ON c.area_number = r.pickup_community_area
    LEFT JOIN (
        SELECT *
        FROM disabilities
        WHERE period_end_year = 2019
        AND segment = 'all'
    ) d 
        ON c.area_number = d.area_number

    
    
""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,name,part,population,area_number,pickup_community_area,avg_trips_before,avg_trips_since,segment,value,std_error,period_start_year,period_end_year,period
0,Rogers Park,Far North Side,52553,1,1,3886,1278,all,10.398003,0.794847,2015,2019,2015-2019
1,West Ridge,Far North Side,78444,2,2,2819,935,all,9.599939,0.700731,2015,2019,2015-2019
2,Uptown,Far North Side,53663,3,3,8394,2365,all,11.831111,1.072494,2015,2019,2015-2019
3,Lincoln Square,Far North Side,42033,4,4,3879,1002,all,6.871001,0.840300,2015,2019,2015-2019
4,North Center,North Side,35603,5,5,4582,1059,all,5.511223,0.629538,2015,2019,2015-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Washington Heights,Far Southwest Side,26635,73,73,1134,492,all,17.531878,1.369235,2015,2019,2015-2019
73,Mount Greenwood,Far Southwest Side,19196,74,74,197,63,all,8.845941,1.067681,2015,2019,2015-2019
74,Morgan Park,Far Southwest Side,21594,75,75,698,294,all,15.250978,1.355346,2015,2019,2015-2019
75,O'Hare,Far North Side,15655,76,76,7938,1504,all,10.950642,1.479030,2015,2019,2015-2019
